In [ ]:
import os
import chromadb
from chromadb.utils import embedding_functions
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader


In [ ]:
chroma_client = chromadb.Client()

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name='all-MiniLM-L6-v2')
metadata_options = {"hnsw:space": "cosine" }

collection = chroma_client.get_or_create_collection(name="data-feminism", metadata=metadata_options, embedding_function=sentence_transformer_ef)

Convert all chapters to embeddings:

In [ ]:
# import os
# folder_path = 'docs'
# for root, _, files in os.walk(folder_path):
#     for file in files:
#         file_path = os.path.join(root, file)
#         print(file_path)

Convert just one chapter to embeddings:

In [ ]:
loader = PyPDFLoader("docs/Ch1.pdf")
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
docs = text_splitter.split_documents(data)
for doc in docs:
    uuid_name = uuid.uuid1()
    collection.add(ids=[str(uuid_name)], documents=doc.page_content)

In [ ]:
def retrieve_context(query_text):
    results = collection.query(query_texts=[query_text],n_results=6)
    return results['documents']

client = OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', 
)

In [ ]:
def chatbot():
  # Create a list to store all the messages for context
  messages = []
  messages.append({"role": "system", "content": "You are a helpful assistant who's been given Data Feminism Chapter as context. When the user asks a question, use the data feminism context to answer the question. If the message is a greeting, just respond normally." })  
  
  while True:
    # list to store all the retrievals from Chroma
    context = []
    
    # Prompt user for input
    message = input("User: ")
    print(f"User: {message}"+'\n')
    
    # Exit program if user inputs "quit"
    if message.lower() == "quit":
      break
    
    #retrieve necessary context
    context = retrieve_context(message)
    # context_string = ', '.join([str(elem) for elem in context])
    
    message_with_context = f''' 
    
    You are a helpful bot who answers concisely. Use this context: 
    {context} 
    
    To answer the question: {message}. 
    
    If you don't know the answer from the context, say you don't know.
    '''
    
    # Add each new message to the list
    messages.append({"role": "user", "content": message_with_context})
    
    response = client.chat.completions.create(
      model="llama3",
      messages=[{"role": "user", "content": message_with_context}]
    )
    
    # Print the response and add it to the messages list
    chat_message = response.choices[0].message.content
    print(f"Bot: {chat_message}"+'\n')
    messages.append({"role": "assistant", "content": chat_message})

In [ ]:
chatbot()